In [1]:
import numpy as np
import xarray as xr
import crocosi.gridop as gop


%matplotlib inline
from matplotlib import pyplot as plt

from itertools import permutations

In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:59315 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 17.18 GB


In [3]:
# generate synthetic data
dims = {'time':3, 'z':5, 'y':20, 'x':10}
vmap = {'time':0., 'z':1., 'y':0., 'x':0.}

In [4]:
ndim = 3
def get_P(ndim):
    return [p for p in list(permutations(dims.keys(),ndim)) if 'z' in p]

#for p in P
# ...
get_P(3)

[('time', 'z', 'y'),
 ('time', 'z', 'x'),
 ('time', 'y', 'z'),
 ('time', 'x', 'z'),
 ('z', 'time', 'y'),
 ('z', 'time', 'x'),
 ('z', 'y', 'time'),
 ('z', 'y', 'x'),
 ('z', 'x', 'time'),
 ('z', 'x', 'y'),
 ('y', 'time', 'z'),
 ('y', 'z', 'time'),
 ('y', 'z', 'x'),
 ('y', 'x', 'z'),
 ('x', 'time', 'z'),
 ('x', 'z', 'time'),
 ('x', 'z', 'y'),
 ('x', 'y', 'z')]

In [5]:
def get_ds(p, chunks=None):
    """ Create a synthetic dataset based on some dimension order p
    """

    # assemble coordinates
    _coords = {d: np.arange(dims[d]) for d in p}
    _coords.update(z_target1d=np.arange(.5,6))
    ds = xr.Dataset(coords=_coords)

    # create data variable and initial grid
    ds['v'] = sum([ds[d]*vmap[d] for d in p])
    ds['z_v'] = ds['z'] + 0.*ds['v']
    # the line above necessarily imposes a dimension order which may vary
    # should loop around all potential dimension order for zv

    # create target grid
    ds['z_target'] = ds['z_target1d']
    # need to vary number of dimensions and their order on the line above
    if chunks:
        ds = ds.chunk(chunks)
    return ds

In [6]:
def test_interp2z_np_3d(ds):
    #print(ds)
    out = gop.interp2z_np_3d(ds['z_target'].values, 
                          (0.*ds['v']+ds['z_v']).values,
                          ds['v'].values, 
                          b_extrap=0, t_extrap=0)
    print('---------')
    print('v shape: {}'.format(list(ds.v.dims)))    
    print('Input shape = {}'.format(ds['v'].values.shape))
    print('Ouput shape = {}'.format(out.shape))
    if out.ndim==3:
        out = out[:,0,0]
    elif out.ndim==2:
        out = out[:,0]
    print(out)
    #hdl = plt.plot(ds['z_target1d'].values, out)
    #plt.grid()
    
test_interp2z_np_3d(get_ds(get_P(3)[6]))
test_interp2z_np_3d(get_ds(get_P(2)[2]))
test_interp2z_np_3d(get_ds(('z',)))

---------
v shape: ['z', 'y', 'time']
Input shape = (5, 20, 3)
Ouput shape = (6, 20, 3)
[0.5 1.5 2.5 3.5 nan nan]
---------
v shape: ['z', 'y']
Input shape = (5, 20)
Ouput shape = (6, 20)
[0.5 1.5 2.5 3.5 nan nan]
---------
v shape: ['z']
Input shape = (5,)
Ouput shape = (6,)
[0.5 1.5 2.5 3.5 nan nan]


In [7]:
get_P(4)

[('time', 'z', 'y', 'x'),
 ('time', 'z', 'x', 'y'),
 ('time', 'y', 'z', 'x'),
 ('time', 'y', 'x', 'z'),
 ('time', 'x', 'z', 'y'),
 ('time', 'x', 'y', 'z'),
 ('z', 'time', 'y', 'x'),
 ('z', 'time', 'x', 'y'),
 ('z', 'y', 'time', 'x'),
 ('z', 'y', 'x', 'time'),
 ('z', 'x', 'time', 'y'),
 ('z', 'x', 'y', 'time'),
 ('y', 'time', 'z', 'x'),
 ('y', 'time', 'x', 'z'),
 ('y', 'z', 'time', 'x'),
 ('y', 'z', 'x', 'time'),
 ('y', 'x', 'time', 'z'),
 ('y', 'x', 'z', 'time'),
 ('x', 'time', 'z', 'y'),
 ('x', 'time', 'y', 'z'),
 ('x', 'z', 'time', 'y'),
 ('x', 'z', 'y', 'time'),
 ('x', 'y', 'time', 'z'),
 ('x', 'y', 'z', 'time')]

In [8]:
def test_interp2z_np(ds):
    #print(ds)
    z_pos = ds.v._get_axis_num('z')
    z_size = ds.dims['z']
    out = gop.interp2z_np(ds['z_target'].values, 
                          (0.*ds['v']+ds['z_v']).values,
                          ds['v'].values, 
                          zdim=(z_pos, z_size),
                          b_extrap=0, t_extrap=0)
    print('---------')
    print('v shape: {}'.format(list(ds.v.dims)))
    print('Input shape = {}'.format(ds['v'].values.shape))
    print('Ouput shape = {}'.format(out.shape))
    if out.ndim==4:
        out = out.swapaxes(0,z_pos)[:,0,0,0]
    elif out.ndim==3:
        out = out[:,0,0]
    elif out.ndim==2:
        out = out[:,0]
    print(out)
    
test_interp2z_np(get_ds(get_P(4)[0]))
test_interp2z_np(get_ds(get_P(4)[6]))
test_interp2z_np(get_ds(get_P(3)[6]))
test_interp2z_np(get_ds(get_P(2)[2]))
test_interp2z_np(get_ds(('z',)))

---------
v shape: ['time', 'z', 'y', 'x']
Input shape = (3, 5, 20, 10)
Ouput shape = (3, 6, 20, 10)
[0.5 1.5 2.5 3.5 nan nan]
---------
v shape: ['z', 'time', 'y', 'x']
Input shape = (5, 3, 20, 10)
Ouput shape = (6, 3, 20, 10)
[0.5 1.5 2.5 3.5 nan nan]
---------
v shape: ['z', 'y', 'time']
Input shape = (5, 20, 3)
Ouput shape = (6, 20, 3)
[0.5 1.5 2.5 3.5 nan nan]
---------
v shape: ['z', 'y']
Input shape = (5, 20)
Ouput shape = (6, 20)
[0.5 1.5 2.5 3.5 nan nan]
---------
v shape: ['z']
Input shape = (5,)
Ouput shape = (6,)
[0.5 1.5 2.5 3.5 nan nan]


In [9]:
def test_interp2z(ds):
    #print(ds)
    out = gop.interp2z(ds['z_target'], ds['v'], 
                          (0.*ds['v']+ds['z_v']),
                          b_extrap=0, t_extrap=0)
    print('---------')
    print('v shape: {}'.format(list(ds.v.dims)))
    print('Input shape = {}'.format(ds['v'].values.shape))
    print('Ouput shape = {}'.format(out.shape))
    if out.ndim==4:
        out = out.swapaxes(0,z_pos)[:,0,0,0]
    elif out.ndim==3:
        out = out[:,0,0]
    elif out.ndim==2:
        out = out[:,0]
    print(out)
    
test_interp2z_np(get_ds(get_P(4)[0]))
test_interp2z_np(get_ds(get_P(4)[6]))
test_interp2z_np(get_ds(get_P(3)[6]))
test_interp2z_np(get_ds(get_P(2)[2]))
test_interp2z_np(get_ds(('z',)))

---------
v shape: ['time', 'z', 'y', 'x']
Input shape = (3, 5, 20, 10)
Ouput shape = (3, 6, 20, 10)
[0.5 1.5 2.5 3.5 nan nan]
---------
v shape: ['z', 'time', 'y', 'x']
Input shape = (5, 3, 20, 10)
Ouput shape = (6, 3, 20, 10)
[0.5 1.5 2.5 3.5 nan nan]
---------
v shape: ['z', 'y', 'time']
Input shape = (5, 20, 3)
Ouput shape = (6, 20, 3)
[0.5 1.5 2.5 3.5 nan nan]
---------
v shape: ['z', 'y']
Input shape = (5, 20)
Ouput shape = (6, 20)
[0.5 1.5 2.5 3.5 nan nan]
---------
v shape: ['z']
Input shape = (5,)
Ouput shape = (6,)
[0.5 1.5 2.5 3.5 nan nan]


In [16]:
def test_interp2z(ds):
    #print(ds)
    out = gop.interp2z(ds['z_target'], ds['v'], 
                       (0.*ds['v']+ds['z_v']),
                       zt_dim='z_target1d',
                       b_extrap=0, t_extrap=0).compute()
    print('---------')
    print('v shape: {}'.format(list(ds.v.dims)))
    print('Input shape = {}'.format(ds['v'].values.shape))
    print('Ouput shape = {}'.format(out.shape))
    z_pos = ds.v._get_axis_num('z')
    if out.ndim==4:
        out = out.data.swapaxes(0,z_pos)[:,0,0,0]
    elif out.ndim==3:
        out = out[:,0,0]
    elif out.ndim==2:
        out = out[:,0]
    print(out)

test_interp2z(get_ds(get_P(4)[0], chunks={'x': 2}))
test_interp2z(get_ds(get_P(4)[0]))
test_interp2z(get_ds(get_P(4)[0], chunks=2))
#test_interp2z(get_ds(get_P(4)[6]))
#test_interp2z(get_ds(get_P(3)[6]))
#test_interp2z(get_ds(get_P(2)[2]))
#test_interp2z(get_ds(('z',)))

---------
v shape: ['time', 'z', 'y', 'x']
Input shape = (3, 5, 20, 10)
Ouput shape = (3, 6, 20, 10)
[0.5 1.5 2.5 3.5 nan nan]
---------
v shape: ['time', 'z', 'y', 'x']
Input shape = (3, 5, 20, 10)
Ouput shape = (3, 6, 20, 10)
[0.5 1.5 2.5 3.5 nan nan]
---------
v shape: ['time', 'z', 'y', 'x']
Input shape = (3, 5, 20, 10)
Ouput shape = (3, 6, 20, 10)
[0.5 1.5 2.5 3.5 nan nan]


In [11]:
ds = get_ds(get_P(4)[0], chunks={'x': 2})

In [12]:
(d for d in ds.v.dims)

<generator object <genexpr> at 0x10312fbd0>

In [13]:
['' if d=='z' else d for d in list(ds.v.dims)]

['time', '', 'y', 'x']

In [14]:
ds

<xarray.Dataset>
Dimensions:     (time: 3, x: 10, y: 20, z: 5, z_target1d: 6)
Coordinates:
  * time        (time) int64 0 1 2
  * z           (z) int64 0 1 2 3 4
  * y           (y) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
  * x           (x) int64 0 1 2 3 4 5 6 7 8 9
  * z_target1d  (z_target1d) float64 0.5 1.5 2.5 3.5 4.5 5.5
Data variables:
    v           (time, z, y, x) float64 dask.array<chunksize=(3, 5, 20, 2), meta=np.ndarray>
    z_v         (z, time, y, x) float64 dask.array<chunksize=(5, 3, 20, 2), meta=np.ndarray>
    z_target    (z_target1d) float64 0.5 1.5 2.5 3.5 4.5 5.5

In [15]:
#xr.align(ds.v, ds.z_v, join='left')
xr.broadcast(ds.v, ds.z_v)

(<xarray.DataArray 'v' (time: 3, z: 5, y: 20, x: 10)>
 dask.array<xarray-v, shape=(3, 5, 20, 10), dtype=float64, chunksize=(3, 5, 20, 2), chunktype=numpy.ndarray>
 Coordinates:
   * time     (time) int64 0 1 2
   * z        (z) int64 0 1 2 3 4
   * y        (y) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
   * x        (x) int64 0 1 2 3 4 5 6 7 8 9,
 <xarray.DataArray 'z_v' (time: 3, z: 5, y: 20, x: 10)>
 dask.array<transpose, shape=(3, 5, 20, 10), dtype=float64, chunksize=(3, 5, 20, 2), chunktype=numpy.ndarray>
 Coordinates:
   * time     (time) int64 0 1 2
   * z        (z) int64 0 1 2 3 4
   * y        (y) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
   * x        (x) int64 0 1 2 3 4 5 6 7 8 9)

def get_ds(p, chunks=None):
    """ Create a synthetic dataset based on some dimension order p
    """

    # assemble coordinates
    _coords = {d: np.arange(dims[d]) for d in p}
    _coords.update(z_target1d=np.arange(.5,6))
    ds = xr.Dataset(coords=_coords)

    # create data variable and initial grid
    #ds['v'] = 0.
    #for d in p:
    #    ds['v'] = ds['v'] + ds[d]*vmap[d]
    #ds['v'] = sum([ds[d]*vmap[d] for d in p])
    #ds['z_v'] = ds['z'] + 0.*ds['v']
    z = xr.DataArray(np.arange(dims['z']), dims=['z']).rename('z_v')
    v = xr.DataArray(np.broadcast_to(z.data[:,None,None],(z.size, 10, 20)),
                     dims=['z', 'y', 'x']).rename('v')
    zt = xr.DataArray(np.arange(.5,6), dims=['z_target1d']).rename('z_target')
    ds = xr.merge([z,v,zt])
    
    # the line above necessarily imposes a dimension order which may vary
    # should loop around all potential dimension order for zv

    # create target grid
    #ds['z_target'] = ds['z_target1d']
    # need to vary number of dimensions and their order on the line above
    if chunks:
        ds = ds.chunk(chunks)
    return ds

ds = get_ds(get_P(4)[0], chunks=2)
ds.mean().compute()

xr.apply_ufunc(np.mean, ds['v'], (1,2), dask='parallelized', output_dtypes=[np.float64])